# Reject Option Classification

**Uses the Reject Option Classification post-processing algorithm from the AI360 toolkit. This algorithm works by giving an unfavorable outcome to privileged groups and a favorable outcome for unprivileged groups in a confidence band around the decision boundary with the highest uncertainty.** https://aif360.readthedocs.io/en/latest/modules/algorithms.html

Based on the example found at https://github.com/IBM/AIF360/blob/master/examples/demo_reject_option_classification.ipynb

In [7]:
%matplotlib inline
import os
import sys
sys.path.insert(0, os.path.abspath('../datasets'))
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider

# Import employment dataset 
from EmploymentDataset import EmploymentDataset
from util import preprocess_employment

In [8]:
# Privileged, unprivileged groups (males are the privileged class, females unprivileged)
privileged_groups = [{"Sex": 1}]
unprivileged_groups = [{"Sex": 0}]

# Load dataset
dataset_orig = preprocess_employment(['Sex'])

# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]


if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

[0. 1.]


In [9]:
# Split into train and test
dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.7], shuffle=True)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True)

In [10]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(34577, 33)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['Sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['Sex', 'Race', 'Education=0', 'Education=1', 'Education=2', 'Education=3', 'Education=4', 'Education=5', 'Education=6', 'Education=7', 'Citizenship=Native', 'Citizenship=Non Native', 'Industry=0', 'Industry=1', 'Industry=2', 'Industry=3', 'Industry=4', 'Industry=5', 'Industry=6', 'Industry=7', 'Industry=8', 'Industry=9', 'Industry=10', 'Industry=11', 'Industry=12', 'Industry=13', 'age_by_decade=10', 'age_by_decade=20', 'age_by_decade=30', 'age_by_decade=40', 'age_by_decade=50', 'age_by_decade=60', 'age_by_decade=>=70']


**Original training data metrics**

In [11]:

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.002010


### Train classifier on original data

In [12]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred

**Obtain scores for validation and test sets**

In [13]:
dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
X_valid = scale_orig.transform(dataset_orig_valid_pred.features)
y_valid = dataset_orig_valid_pred.labels
dataset_orig_valid_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

### Find the optimal parameters from the validation set

**Best theroshold for classification (no fairness)**

In [14]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_valid_pred.scores > class_thresh
    dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
    dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label
    
    classified_metric_orig_valid = ClassificationMetric(dataset_orig_valid,
                                             dataset_orig_valid_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_valid.true_positive_rate()\
                       +classified_metric_orig_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 0.6058
Optimal classification threshold (no fairness constraints) = 0.9405


**Estimate optimal parameters for the ROC method**

In [15]:

ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_valid, dataset_orig_valid_pred)

In [16]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.9405
Optimal ROC margin = 0.0012


### Predictions from validation set

In [17]:
# Metrics for the test set
fav_inds = dataset_orig_valid_pred.scores > best_class_thresh
dataset_orig_valid_pred.labels[fav_inds] = dataset_orig_valid_pred.favorable_label
dataset_orig_valid_pred.labels[~fav_inds] = dataset_orig_valid_pred.unfavorable_label

display(Markdown("#### Validation set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_valid_bef = compute_metrics(dataset_orig_valid, dataset_orig_valid_pred, 
                unprivileged_groups, privileged_groups)

#### Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.6058
Statistical parity difference = 0.0099
Disparate impact = 1.0210
Average odds difference = 0.0260
Equal opportunity difference = 0.0056
Theil index = 0.6538


In [18]:
# Transform the validation set
dataset_transf_valid_pred = ROC.predict(dataset_orig_valid_pred)

display(Markdown("#### Validation set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_valid_aft = compute_metrics(dataset_orig_valid, dataset_transf_valid_pred, 
                unprivileged_groups, privileged_groups)

#### Validation set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.6063
Statistical parity difference = 0.0360
Disparate impact = 1.0801
Average odds difference = 0.0565
Equal opportunity difference = 0.0310
Theil index = 0.6649


In [20]:
# Testing: Check if the metric optimized has not become worse
assert np.abs(metric_valid_aft[metric_name]) <= np.abs(metric_valid_bef[metric_name])

AssertionError: 

### Predictions from Test set

In [21]:
# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.5994
Statistical parity difference = -0.0252
Disparate impact = 0.9490
Average odds difference = -0.0212
Equal opportunity difference = -0.0261
Theil index = 0.6487


In [22]:
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.5974
Statistical parity difference = 0.0012
Disparate impact = 1.0025
Average odds difference = 0.0019
Equal opportunity difference = 0.0008
Theil index = 0.6590


# Summary

### Fairness Metric: Statistical parity difference, Accuracy Metric: Balanced accuracy

**Performance**

| Dataset | Sex (Acc-Bef) | Sex (Acc-Bef) | Sex (Fair-Bef) | Sex (Fair-Aft) |
|---------|---------------|---------------|----------------|----------------|
| Valid   | 0.6058        | 0.6063        | -0.0252        | 0.0360         |
| Test    | 0.5994        | 0.5974        | -0.0252        | 0.0012         |

**Optimal parameters**

|                                                      | Optimum |
|-----------------------------------------------------:|---------|
| Sex (Class. thresh.)                                 | 0.9405  |
| Sex (Class. thresh. - fairness)                      | 0.9405  |
| (ROC margin - fairness) | 0.0012  |

### Fairness Metric: Average odds difference, Accuracy Metric: Balanced accuracy


**Performance**

## To Do (These are just sample values)

| Dataset | Sex (Acc-Bef) | Sex (Acc-Bef) | Sex (Fair-Bef) | Sex (Fair-Aft) |
|---------|---------------|---------------|----------------|----------------|
| Valid   | 0.6058        | 0.6063        | -0.0252        | 0.0360         |
| Test    | 0.5994        | 0.5974        | -0.0252        | 0.0012         |